MIT License

Copyright (c) Microsoft Corporation. All rights reserved.

This notebook is adapted from Microsoft Learning mslearn-dp100 

Copyright (c) 2021 PyLadies Amsterdam, Alyona Galyeva

# Generate batch data

In [10]:
import os
import pandas as pd
import numpy as np
from azureml.core import Workspace, Dataset

In [11]:
WORKDIR = os.getcwd()

In [12]:
# Create a folder
BATCH_FOLDER = 'capstone-batch-data'
os.makedirs(BATCH_FOLDER, exist_ok=True)
print("Folder created!")

Folder created!


In [13]:
# get the workspace from config.json
ws = Workspace.from_config()
# get the datastore to upload our data
datastore = ws.get_default_datastore()

In [26]:
# retrieve test dataset from Azure Datastore
test_ds = Dataset.get_by_name(ws, name="capstone_data_processed")
test = test_ds.to_pandas_dataframe()
test.head()

,data_index_,workday_temperature,workday_solar_ghi,workday_f_quarter_0,workday_f_quarter_1,workday_f_quarter_2,workday_f_quarter_3,workday_f_quarter_4,workday_f_quarter_5,weekend_temperature,...,weekend_f_quarter_3,weekend_f_quarter_4,weekend_f_quarter_5,load,load_actuals_mw_lag1,load_actuals_mw_lag2,load_actuals_mw_lag3,load_actuals_mw_lag4,load_actuals_mw_lag5,load_actuals_mw_lag6
0,2020-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,274.989655,...,0.991445,0.19509,0.980785,95.756328,95.756328,95.756328,95.756328,95.756328,95.756328,95.756328
1,2020-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,274.423157,...,0.991445,0.19509,0.980785,91.506670,95.756328,95.756328,95.756328,95.756328,95.756328,95.756328
2,2020-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,274.266571,...,0.991445,0.19509,0.980785,87.394102,91.506670,95.756328,95.756328,95.756328,95.756328,95.756328
3,2020-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,274.130737,...,0.991445,0.19509,0.980785,84.923467,87.394102,91.506670,95.756328,95.756328,95.756328,95.756328
4,2020-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,273.969421,...,0.991445,0.19509,0.980785,84.112076,84.923467,87.394102,91.506670,95.756328,95.756328,95.756328


In [1]:
X =(test.set_index(['data_index_'])
     .loc['2021-05-31 00:00:00':'2021-06-06 00:00:00']
     .reset_index(drop=True)
     .drop(['load'], axis=1)
     .to_numpy()
     )

NameError: name 'test' is not defined

In [ ]:
# pick up 1 week from Monday 31-07-2017 to Sunday 06-08-2017, drop target and timestamp, convert to numpy arrow
# X = test[9480:9648]
# X = X.drop(['demand', 'timeStamp'], axis=1)
# X = X.to_numpy()

In [39]:
# Save each sample as a separate file
print("Saving files...")
for i in range(len(X)):
    fname = str(i+1) + '.csv'
    X[i].tofile(os.path.join(BATCH_FOLDER, fname), sep=",")
print("files saved!")

Saving files...
files saved!


In [41]:
datastore.upload(src_dir="capstone-batch-data", target_path="capstone-batch-data", overwrite=True, show_progress=True)

Uploading an estimated of 145 files
Uploading capstone-batch-data\1.csv
Uploaded capstone-batch-data\1.csv, 1 files out of an estimated total of 145
Uploading capstone-batch-data\10.csv
Uploaded capstone-batch-data\10.csv, 2 files out of an estimated total of 145
Uploading capstone-batch-data\100.csv
Uploaded capstone-batch-data\100.csv, 3 files out of an estimated total of 145
Uploading capstone-batch-data\101.csv
Uploaded capstone-batch-data\101.csv, 4 files out of an estimated total of 145
Uploading capstone-batch-data\102.csv
Uploaded capstone-batch-data\102.csv, 5 files out of an estimated total of 145
Uploading capstone-batch-data\103.csv
Uploaded capstone-batch-data\103.csv, 6 files out of an estimated total of 145
Uploading capstone-batch-data\104.csv
Uploaded capstone-batch-data\104.csv, 7 files out of an estimated total of 145
Uploading capstone-batch-data\105.csv
Uploaded capstone-batch-data\105.csv, 8 files out of an estimated total of 145
Uploading capstone-batch-data\106.

$AZUREML_DATAREFERENCE_c2164ed961d54ac8b81baf9b05d02b6d

In [43]:
# Register a dataset for the input data
batch_data_set = Dataset.File.from_files(path=(datastore, 'capstone-batch-data/'), validate=False)
try:
    batch_data_set = batch_data_set.register(workspace=ws, 
                                             name='capstone-batch-data',
                                             description='batch data capstone batch forecast',
                                             create_new_version=True)
except Exception as ex:
    print(ex)

print("Done!")

Done!
